# 07-04 Multi-Agent 系统

多个 AI Agent 协作完成复杂任务。

## 1. 基础 Agent 类

In [ ]:
class Agent {
  constructor(name, role, tools = []) {
    this.name = name;
    this.role = role;
    this.tools = tools;
    this.memory = [];
  }
  
  async think(task) {
    // 子类实现具体逻辑
    throw new Error('Not implemented');
  }
  
  async useTool(toolName, params) {
    const tool = this.tools.find(t => t.name === toolName);
    if (!tool) throw new Error(`Tool ${toolName} not found`);
    return await tool.execute(params);
  }
}

## 2. 多 Agent 协作示例

In [ ]:
// 研究员 Agent - 负责信息收集
class ResearcherAgent extends Agent {
  async think(topic) {
    return `关于 ${topic} 的研究结果：...`;
  }
}

// 写手 Agent - 负责内容生成
class WriterAgent extends Agent {
  async think(research) {
    return `基于研究内容撰写的文章：${research}`;
  }
}

// 编辑 Agent - 负责审核修改
class EditorAgent extends Agent {
  async think(draft) {
    return `修改建议：${draft}`;
  }
}

// 协作流程
async function collaborativeWriting(topic) {
  const researcher = new ResearcherAgent('研究员', 'research');
  const writer = new WriterAgent('写手', 'write');
  const editor = new EditorAgent('编辑', 'edit');
  
  // 1. 研究
  const research = await researcher.think(topic);
  
  // 2. 写作
  const draft = await writer.think(research);
  
  // 3. 编辑
  const final = await editor.think(draft);
  
  return final;
}

## 3. Agent 通信机制

In [ ]:
class AgentBus {
  constructor() {
    this.agents = new Map();
    this.messages = [];
  }
  
  register(agent) {
    this.agents.set(agent.name, agent);
    agent.bus = this;
  }
  
  async send(from, to, message) {
    const msg = {
      id: Date.now(),
      from,
      to,
      content: message,
      timestamp: new Date()
    };
    this.messages.push(msg);
    
    const target = this.agents.get(to);
    if (target) {
      await target.receive(msg);
    }
  }
  
  broadcast(from, message) {
    for (const [name, agent] of this.agents) {
      if (name !== from) {
        this.send(from, name, message);
      }
    }
  }
}

## 练习

1. 实现一个客服多 Agent 系统（接待→问题分类→解答→回访）
2. 设计 Agent 间的任务委托机制
3. 查看 OpenClaw 的 Agent 实现